In [4]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import requests
import json
import csv
import os
from datetime import datetime 

In [12]:
import os
files = os.listdir("../../data/")
files.sort()
files

['BNBUSDT']

In [6]:
def get_figure_for_df(df, name):
   return go.Candlestick(
          name= name,
          x=df['open time'],
          open=df['open'],
          high=df['high'],
          low=df['low'],
          close=df['close']
      )

def get_file(file):
    df = pd.read_csv(f'../../data/{file}')
    df['open time'] = pd.to_datetime(df['open time'], unit='ms')
    std = df['high'].std()
    mean = df['high'].mean()
    df['open'] = (df['open'] - mean) / std
    df['high'] = (df['high'] - mean) / std
    df['low'] = (df['low'] - mean) / std
    return df

In [17]:
def candlestick(files):
  first = get_file(files[0])
  max_ = first['high'].max()
  min_ = first['low'].min()
  event_time = datetime.strptime("2020-04-13T09:53:13.000Z", '%Y-%m-%dT%H:%M:%S.%fZ') #first.loc[250, 'open time']

  figures = []
  for file in files:
      df = get_file(file)
      max_ = max(max_, df['high'].max())
      min_ = min(min_, df['low'].min())
      print(df.loc[250])
      figure = get_figure_for_df(df, file)
      figures.append(figure)

  fig = go.Figure(data=figures)
  fig.update_layout(
    title= {
     'text': ':D',
     'y':0.9,
     'x':0.5,
     'xanchor': 'center',
     'yanchor': 'top'
    },
   font=dict(
    family="Courier New, monospace",
    size=20,
    color="#7f7f7f"
  ))
  fig.add_shape(
    # Line Vertical
    dict(
        type="line",
        x0=event_time,
        y0=min_,
        x1=event_time,
        y1=max_,
        line=dict(
            color="RoyalBlue",
            width=1
        )
  ))
  fig.show()
 
 
candlestick(files[13:15])

open time                       2020-04-13 14:04:00
open                                        14.3348
high                                        14.3459
low                                         14.2578
close                                       14.2578
volume                                      12579.6
close time                            1586786699999
quote asset volume                           180203
number of trades                                390
taker buy base asset volume                 6311.78
taker buy quote asset volume                  90469
ignore                                            0
Name: 250, dtype: object
open time                       2020-04-13 14:04:00
open                                          273.3
high                                          273.3
low                                           273.3
close                                         273.3
volume                                            0
close time                            1